# 10. Jupyter i Excel

U ovoj lekciji ćemo govoriti o:
1. odnos Jupytera i Microsofta Excel-a;
2. učitavanje tablica iz Excel datoteka; i
3. pisanje tablica u Excel datoteke.

## 10.1. Zašto Jupyter i zašto Excel

Microsoft Excel najpopularniji je program za proračunske tablice na svijetu. Svoju popularnost duguje činjenici da je tablica na kojoj radite točno tamo ispred vas, možete je vidjeti, možete kliknuti na ćeliju i unijeti vrijednost ili formulu. Tipičan je predstavnik filozofije _Ono što vidiš, je ono što dobiješ_ . Dakle, zašto smo se odlučili fokusirati ovaj tečaj na Jupyter?

* _Cijena._ Microsoft Excel je komercijalni proizvod - košta novac. Za razliku od tog Python, sve njegove biblioteke koje prate i Jupyter kao interaktivno okruženje su _besplatni_.

* _Jasno vidljivi postupci._ Obrada podataka u programu Microsoft Excell sastoji se od unošenja formula u ćelije. Ako radite s tablicom s zamršenim odnosima između stanica izraženih mnogim kompliciranim formulama raspoređenim po cijelom radnom listu, ubrzo postaje gotovo nemoguće pratiti protok informacija i, što je još važnije, razumjeti, ispraviti pogrešku i poboljšati postupak. S druge strane, ako je obrada izražena programskim jezikom (kao što je Python), mi gubimo _ono što vidite je ono što dobivate u Excelu_ , ali puno više dobivamo na čitljivosti koda. Jasni postupak smješten na jednom mjestu (Jupyterova ćelija ili Python datoteka) i kodiran jednostavnim i ekspresivnim programskim jezikom lako se može provjeriti za pogreške, nadograditi i podijeliti.

* _Fleksibilnost._ Microsoft Excel prikladan je za obradu relativno malih tablica tako da se lako mogu uklopiti na nekoliko zaslona računala. Jednom kada se nađete u položaju u kojem morate obraditi ogromne tablice s tisućama redaka i stupaca, prednosti skriptnih jezika postaju očite. Štoviše, svaka Python distribucija dolazi s velikim brojem biblioteka u kojima je većina standardnih algoritama za obradu podataka su već implementirana.

Korištenje jasno vidljivih postupaka koji se ne stapaju s podacima koji se analiziraju najefikasniji je način obrade podataka. To je kamen temeljac svakog pristupa moderne obrade podataka.

## 10.2. Učitavanje tablica iz lokalnih Excel datoteka

Svaki Excel dokument sastoji se od nekoliko radnih listova. Svaki je radni list tablica kojoj se može pristupiti kroz njegovo ime:

![Worksheets](pics/Excel.jpg)

Budući da je Microsoft Excel najpopularniji program za proračunske tablice na svijetu, biblioteka _pandas_ ima način za učitavanje radnog lista Excelovog dokumenta u DataFrame. Ako se Excel dokument sastoji od više radnih listova, moramo ga učitati kao nekoliko DataFrames - jedan DataFrame po radnom listu.

Na primjer, datoteka podataka _data/Additives.xlsx_ ima jedan radni list "E-brojevi" koji izravno postavljamo u DataFrame:

In [ ]:
import pandas as pd
aditivi = pd.read_excel("podaci/Aditivi.xlsx", sheet_name="Aditivi")

Ova datoteka sadrži neke osnovne podatke o aditivima koji su tvari koje se koriste u prehrambenoj industriji za očuvanje hrane ili poboljšanje njihove boje i ukusa. Pogledajmo na stol:

In [ ]:
aditivi.head(15)

Stupci koji su bili prazni u Excelovoj tablici dobivaju posebnu NaN vrijednost, koja označava "nije broj". Budući da u našem slučaju ove ćelije predstavljaju komentare, želimo da prazne ćelije ostanu prazne. Dakle, ponovno ćemo uložiti tablicu, ali ovaj put uputimo sustav da se ne žali na prazne ćelije:

In [ ]:
aditivi = pd.read_excel("podaci/Aditivi.xlsx", sheet_name="Aditivi", na_filter=False)
aditivi.head(15)

Opcija `na_filter=False` upućuje funkciju `read_excel` da "isključi umjetnu inteligenciju" i prazne ćelije ostavi praznima. Napravimo frekvencijsku analizu na temelju štetnosti aditiva.

In [ ]:
aditivi["Štetnost"].value_counts()

Sada ćemo filtrirati tablicu kako bismo izdvojili aditive koji mogu izazvati rak:

In [ ]:
aditivi[aditivi.Napomena == "može izazvati rak"]

Na kraju, nabrojimo aditive koji su označeni kao OPASNI ili mogu izazvati rak. Da bismo to učinili, moramo kombinirati dva kriterija filtriranja:

    Comment == "may cause cancer"  or  Status == "DANGEROUS" (or both)

Kad moramo kombinirati dva kriterija tako da je red uključen u filtriranu tablicu ako je zadovoljan barem jedan od kriterija, koristimo `|` priključak:

In [ ]:
aditivi[(aditivi.Napomena == "može izazvati rak") | (aditivi.Štetnost == "IZUZETNO OPASAN")]

## 10.3. Pisanje tablica u Excel datoteke

Svaka tablica se može zapisati u Excel datoteku baš onako kako smo ih koristili u _CSV_ datotekama. Jedina je razlika što umjesto funkcije `to_csv` pozivamo funkciju `to_excel`. Na primjer, napravimo tablicu s popisom aditiva koji su označeni opasnim ili mogu uzrokovati rak:

In [ ]:
opasni_aditivi = aditivi[(aditivi.Napomena == "može izazvati rak") | (aditivi.Štetnost == "IZUZETNO OPASAN")]

i dopustite nam da tablicu napišemo u datoteku Excel:

In [ ]:
opasni_aditivi.to_excel("podaci/opasni_aditivi.xlsx", encoding="utf-8")

Pogledajmo Excel datoteku koju smo upravo stvorili

![Excel file](pics/BadAdditives.jpg)

Vidimo da je sustav također napisao stupac indeksa tablice, koji je u ovom slučaju samo popis besmislenih cjelobrojnih brojeva. Da biste ga se riješili, ponovo ćemo napisati tablicu, ali ovaj put koristeći opciju `index=False`:

In [ ]:
opasni_aditivi.to_excel("podaci/opasni_aditivi.xlsx", encoding="utf-8", index=False)

Nova datoteka izgleda ovako:

![Excel file](pics/BadAdditives2.jpg)

To je upravo ono što smo željeli.

## 10.4. Zadaci

Zadatke riješite u Jupyteru

**Zadatak 1.** Podaci datoteke `data/CS201.xlsx` imaju pregled bodova grupe učenika informatike 201. Podaci su stvarni, pa je tablica anonimna.

_(a)_ Učitajte ovu tablicu u DataFrame i pogledajte prvih nekoliko redaka da biste razumjeli strukturu tablice ("Hnn" označava "domaći zadatak nn", "Cn" označava "kolokvij n", "WE" stoji za "pisani dio ispita", a "OE" znači "usmeno ispitivanje").

_(b)_ Tablicu indeksirajte s "StudentID".

_(c)_ Izračunajte prosječnu ocjenu na svakom kolokviju (stupci "C1", "C2" i "C3").

_(d)_ Dodajte novi stupac "Prosj." i za svakog učenika izračunajte prosječnu ocjenu i upišite je u odgovarajuću ćeliju.

_(e)_ Dodajte novi stupac "FinalGrade" i za svakog učenika izračunajte konačnu ocjenu na temelju prosječne ocjene koristeći sljedeću funkciju:

In [ ]:
def final_grade(avg):
    if avg >= 4.50:
        return 5
    elif avg >= 3.50:
        return 4
    elif avg >= 2.50:
        return 3
    elif avg >= 1.50:
        return 2
    else:
        return 1

_(f)_ Napišite novu tablicu u podatke datoteke Excel _data/CS201-FinalGrades.xlsx_

**Exercise 2.** _Eurostat_ je službena europska agencija zadužena za statističke analize različitih podataka važnih za razvoj Europske unije. Svi podaci koje Eurostat prikuplja javno su dostupni na sljedećoj poveznici: https://ec.europa.eu/eurostat/data/database

Podaci u datoteci _data/EUProjPop.xlsx_ sadrže projekciju stanovništva svake od zemalja EU-a 2080. Tablica ima dva radna lista: Bazna linija koja sadrži predviđeno stanovništvo zemalja EU-a i Migracije koja sadrži predviđeno stanovništvo zemalja EU-a u slučaju pojačana migracija.

_(a)_ Učitajte ta dva radna lista u DataFrames (Baseline i Migration) i prikažite nekoliko redaka svake tablice da biste razumjeli strukturu tablica.

_(b)_ Dodajte novi redak "EU" svakoj tablici i izračunajte projiciranu populaciju cijele unije za svaku godinu.

_(c)_ Dodajte novi redak u tablicu Migracije i izračunajte projiciranu migraciju za svaku godinu (oduzmite redni EU u osnovnoj tablici od retka EU u tablici Migracije).

_(d)_ Vizualizirajte projiciranu migraciju linijskim grafikonom.

_(e)_ Dodajte novi redak "EU-UK" u Tablicu osnovne i izračunajte predviđeno stanovništvo EU-a bez Velike Britanije.

_(f)_ Zapišite dva DataFrames-a podacima _data/EU-UK.xlsx_ and _data/EU-Migration.xlsx_

**Zadatak 3.** Podaci datoteke `data/Cricket.xlsx` sadrže podatke o najboljim igračima kriketa u povijesti kriketa.

_(a)_ Učitajte ovu tablicu u DataFrame i indeksirajte je u stupcu "Player".

_(b)_ Dodajte novi stupac "YP" (igrane godine) u tablicu i izračunajte broj godina aktivnog igranja za svakog igrača (oduzmite stupac "Od" iz stupca "Do").

_(c)_ U tablicu dodajte novi stupac "ARY" (Prosječni trci po godini) i za svakog igrača izračunajte prosječni broj trčanja godišnje (ARY = Runs / YP).

_(d)_ Tablicu sortirajte po "ARY" u silaznom orederu i prikažite prvih 25 redaka tablice. U kojem stoljeću je većina od najboljih 25 igrača igrala aktivno? Što mislite zašto?